In [1]:
import pandas as pd
import numpy as np
import random
import json
import redis

import openai
# openai.api_key = 

## Fine-tuning

In [2]:
!pip install --upgrade openai

In [3]:
!openai tools fine_tunes.prepare_data -f sampletables.jsonl -q

Analyzing...

- Your file contains 10 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix `.\n\n###\n\n`
- All prompts start with prefix `A `
- All completions end with suffix `\n###`

No remediations found.

You can use your file for fine-tuning:
> openai api fine_tunes.create -t "sampletables.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string `.\n\n###\n\n` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=["\n###"]` so that the generated texts ends at the expected place.
Once your model starts training, it'll approximately take 2.58 minutes to train a `curie` model, and less for `ada` and `babbage`. Queue will approximately take half an hour per job ahead of you.


In [11]:
!openai api fine_tunes.create -t "sampletables.jsonl"

Error: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details, or email support@openai.com if you have any questions.


In [5]:
# caching
redis_client = redis.Redis(host = 'localhost', port = 6379, db = 0)

In [6]:
df = pd.read_csv('results/bioguide.csv')
df = df[['usCongressBioId', 'profileText']]
df.head()

,usCongressBioId,profileText
0,W000374,A Representative from Kansas; born near Fairfi...
1,L000226,"A Representative from Florida; born in Selma, ..."
2,F000260,"A Representative from Michigan, Vice President..."
3,M000777,A Representative and a Senator from Texas; bor...
4,W000724,"A Representative from Ohio; born in Johnstown,..."


In [7]:
sample_number = 10
bio_sample = df.sample(n=sample_number, random_state=123)
bio_sample.to_csv('results/samplebios.csv', index=False)

In [8]:
def gen_table(id, bio, update=None):
    """
    Takes in a bio, let chatGPT generate the table, and outputs the table
    """
    table_key = f"politician_{id}"
    table = redis_client.get(table_key)

    if update:
        table = None

    if not table:
        print('Could not find table in cache. Retrieving from Chat GPT API...')
        prompt = prompt_pre + bio
        completion = openai.ChatCompletion.create(
            model='gpt-3.5-turbo', 
            messages=[{'role':'user', 'content': prompt}]
        )

        table = completion.choices[0].message.content
        redis_client.set(table_key, json.dumps(table))
    
    else:
        print('Found table in cache, serving from redis...')
        table = json.loads(table)

    return table

In [9]:

query = prompt_pre + sample_bio

NameError: name 'prompt_pre' is not defined

In [ ]:
completion = openai.ChatCompletion.create(
    model='gpt-3.5-turbo', 
    messages=[{'role':'user', 'content': query}]

)
print(completion)

In [ ]:
df = pd.read_csv('results/bioguide.csv')
df.head()

In [ ]:
ids = df.usCongressBioId.tolist()
sample_ids = random.sample(ids, 10)
sample_bios = [df.loc[df['usCongressBioId'] == id].iloc[0]['profileText'] for id in sample_ids]

In [ ]:
tables = []
for i in range(len(sample_ids)):
    table = gen_table(sample_ids[i], sample_bios[i])
    tables.append(table)

for table in tables:
    print(table)

In [ ]:
sample = ''
for i in range(len(tables)):
    sample += sample_ids[i] + '\n'
    sample += sample_bios[i] + '\n'
    sample += tables[i] + '\n'
print(sample)


In [ ]:
len(tables)

In [ ]:
with open('sampletables.txt', 'w') as f:
    f.write(sample)